### 🛠️ Configuração de device

In [2]:
import torch

n_devices = torch.cuda.device_count()

device_ = "cpu" if not n_devices else "cuda"

### 🔊 Extração do áudio

In [3]:
import ffmpeg, imageio_ffmpeg
import os

In [4]:
path_video = "/home/elton/Projetos/Translater_Video/teste"
name_video = "1 - Preparations.mp4"

name_audio_out = "audio_completo.wav"
path_out = "/home/elton/Projetos/Translater_Video/teste/resultado_teste"

In [5]:
def extrair_audio(path_in, path_out, name_video, name_video_out):
    input_ = os.path.join(path_in, name_video)
    output_ = os.path.join(path_out, name_video_out)
    path_ffmpeg = imageio_ffmpeg.get_ffmpeg_exe()

    # --- Extração de áudio  ---
    #ss -> Inicio, to -> Fim, ac -> mono, ar -> sample_rate, sample_fmt -> bit
    try:
        (
            ffmpeg
            .input(input_)
            .output(output_, vn=None,)
            .run(cmd=path_ffmpeg, overwrite_output=True, quiet=True)
        )
        print(f'Extração de áudio concluída: {output_}')
    except ffmpeg.Error as erro:
        print(erro)

In [6]:
extrair_audio(path_video, path_out, name_video, name_audio_out)

Extração de áudio concluída: /home/elton/Projetos/Translater_Video/teste/resultado_teste/audio_completo.wav


### 🎤 Detecção de fala

In [7]:
from pyannote.audio import Pipeline
import math
import json

/home/elton/Projetos/Translater_Video/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/elton/Projetos/Translater_Video/.venv/lib/python3.10/site-packages/pyannote/audio/core/io.py:212: UserWarning: torchaudio._backend.list_audio_backends has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch/audio/issues/3902 for more information. It will be removed from the 2.9 release. 
  torchaudio.list_audio_backends()


In [8]:
# Carregando o modelo
model = Pipeline.from_pretrained(
    "pyannote/speaker-diarization-3.1",
    use_auth_token=token)

model.to(torch.device(device_))

/home/elton/Projetos/Translater_Video/.venv/lib/python3.10/site-packages/speechbrain/utils/torch_audio_backend.py:57: UserWarning: torchaudio._backend.list_audio_backends has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch/audio/issues/3902 for more information. It will be removed from the 2.9 release. 
  available_backends = torchaudio.list_audio_backends()


In [ ]:
# Função para usar o modelo de detecção de áudio
def timer_speaks(path, model):
    result = model(path)
    return result

In [10]:
path_audio_for_timer_speaks = os.path.join(path_out, name_audio_out)
result_timer_speaks = timer_speaks(path_audio_for_timer_speaks, model)

AttributeError: 'SpeakerDiarization' object has no attribute 'stream'

In [ ]:
list_time_speaker = []
# Parametros do laço foram retirados da documentação do modelo, apenas fiz adaptações
for speaker, _, id_speaker in result_timer_speaks.itertracks(yield_label=True):
    list_time_speaker.append({
        'start': round(speaker.start, 2),
        'end': round(speaker.end, 2) 
    })

In [ ]:
list_time_speaker_reduzido = []
i = 0

while i < len(list_time_speaker):
    start = list_time_speaker[i]['start']
    end = list_time_speaker[i]['end']
    a = i

    while a + 1 < len(list_time_speaker) and math.isclose(end, list_time_speaker[a+1]['start'], abs_tol=1.0):
        end = list_time_speaker[a+1]['end']  # atualiza o fim
        a += 1  # avança para o próximo

    # Adiciona o intervalo unido
    list_time_speaker_reduzido.append({'start':start, 'end': end})
    i = a + 1  # pula para o próximo intervalo que não foi unido

In [23]:
print(f'{len(list_time_speaker)} | {list_time_speaker}')
print(f'{len(list_time_speaker_reduzido)} | {list_time_speaker_reduzido}')

15 | [{'start': 6.92, 'end': 27.96}, {'start': 28.52, 'end': 29.87}, {'start': 30.9, 'end': 34.52}, {'start': 35.57, 'end': 54.94}, {'start': 50.39, 'end': 50.71}, {'start': 55.79, 'end': 73.79}, {'start': 75.16, 'end': 85.89}, {'start': 88.59, 'end': 106.28}, {'start': 108.86, 'end': 110.95}, {'start': 113.09, 'end': 125.56}, {'start': 127.42, 'end': 138.68}, {'start': 139.44, 'end': 145.85}, {'start': 147.03, 'end': 159.74}, {'start': 162.1, 'end': 165.61}, {'start': 168.97, 'end': 191.6}]
13 | [{'start': 6.92, 'end': 29.87}, {'start': 30.9, 'end': 34.52}, {'start': 35.57, 'end': 54.94}, {'start': 50.39, 'end': 50.71}, {'start': 55.79, 'end': 73.79}, {'start': 75.16, 'end': 85.89}, {'start': 88.59, 'end': 106.28}, {'start': 108.86, 'end': 110.95}, {'start': 113.09, 'end': 125.56}, {'start': 127.42, 'end': 145.85}, {'start': 147.03, 'end': 159.74}, {'start': 162.1, 'end': 165.61}, {'start': 168.97, 'end': 191.6}]


In [ ]:
# Salvando o log
path_log = os.path.join(path_out, 'log', 'log.json')

with open(path_log, 'w', encoding='utf-8') as file:
    file.write(json.dumps(list_time_speaker_reduzido, ensure_ascii=False, indent=2))

### 🎼 Extrair audios

- Levando em consideração os tempos de áudio

In [24]:
def extrair_audio_f(path_in, path_out, name_audio, name_audio_out:str, list_times):
    input_ = os.path.join(path_in, name_audio)
    path_ffmpeg = imageio_ffmpeg.get_ffmpeg_exe()

    # --- Extração de áudio  ---
    for index, i in enumerate(list_times):
        
        output_ = os.path.join(path_out, f'{index}.wav')
        try:
            (
                ffmpeg
                .input(input_, ss=i['start'], to=i['end'])
                .output(output_)
                .run(cmd=path_ffmpeg, overwrite_output=True, quiet=True)
            )
            print(f'Extração de áudio concluída: {output_}')
            
            list_time_speaker_reduzido[index]['path_audio'] = output_
            
        except:
            print('Erro no ffmpeg')
    #ss -> Inicio, to -> Fim, ac -> mono, ar -> sample_rate, sample_fmt -> bit
     

In [25]:
path_out  = os.path.join("/home/elton/Projetos/Translater_Video/teste/resultado_teste", 'audios','separados_01')
path_in = "/home/elton/Projetos/Translater_Video/teste/resultado_teste"

In [26]:
extrair_audio_f(path_in, path_out, name_audio_out, 'audios', list_time_speaker_reduzido)

Extração de áudio concluída: /home/elton/Projetos/Translater_Video/teste/resultado_teste/audios/separados_01/0.wav
Extração de áudio concluída: /home/elton/Projetos/Translater_Video/teste/resultado_teste/audios/separados_01/1.wav
Extração de áudio concluída: /home/elton/Projetos/Translater_Video/teste/resultado_teste/audios/separados_01/2.wav
Extração de áudio concluída: /home/elton/Projetos/Translater_Video/teste/resultado_teste/audios/separados_01/3.wav
Extração de áudio concluída: /home/elton/Projetos/Translater_Video/teste/resultado_teste/audios/separados_01/4.wav
Extração de áudio concluída: /home/elton/Projetos/Translater_Video/teste/resultado_teste/audios/separados_01/5.wav
Extração de áudio concluída: /home/elton/Projetos/Translater_Video/teste/resultado_teste/audios/separados_01/6.wav
Extração de áudio concluída: /home/elton/Projetos/Translater_Video/teste/resultado_teste/audios/separados_01/7.wav
Extração de áudio concluída: /home/elton/Projetos/Translater_Video/teste/resulta